### purpose

run nuisance envs on multivariate sims

In [1]:
from pythonimports import *

import MVP_summary_functions as mvp

t0 = dt.now()  # notebook timer

nuis_dir = makedir('/home/b.lind/offsets/run_20220919_nuisance/tutorial')

gf_training_dir = makedir(f'{nuis_dir}/gradient_forests/training/training_files')
gf_shdir = makedir(gf_training_dir.replace('_files', '_shfiles'))
gf_outdir = makedir(gf_shdir.replace('_shfiles', '_outfiles'))

mvp.latest_commit()
session_info.show()

#########################################################
Today:	May 31, 2023 - 16:43:44
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit c721bd0d89c1e150e17a4e0739c26e4296405233  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Wed May 10 15:13:03 2023 -0400

Current commit of MVP_offsets:
commit 060db40dff8355d2087deab28bd695fcccee8f49  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu May 18 12:50:56 2023 -0400
#########################################################



In [2]:
gf_training_dir

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_files'

In [3]:
gf_shdir

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_shfiles'

In [4]:
gf_outdir

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_outfiles'

# get nuis data

created in 02_analysis/07_experiments/03_nuisance_shuffle/01_train_shuffle_nuisance_gf.ipynb - this added nuis_envs to real environmental data. The real environmental data was created in `/home/b.lind/offsets/tutorial_shuffle_run_20220919`. the 'shuffling' was just for the allele freqs.

In [5]:
# get the files
envdir = '/home/b.lind/offsets/tutorial_shuffle_nuisance_run_20220919/gradient_forests/training/training_files'

envfiles = fs(envdir, exclude='-gt-')

envfiles

['/home/b.lind/offsets/tutorial_shuffle_nuisance_run_20220919/gradient_forests/training/training_files/tutorial_envfile_GFready_pooled.txt',
 '/home/b.lind/offsets/tutorial_shuffle_nuisance_run_20220919/gradient_forests/training/training_files/tutorial_rangefile_GFready_pooled.txt']

In [6]:
# read them in and take a look (rangefiles don't have index)
envdfs = {}
for envfile in envfiles:
    if 'range' in envfile:
        index_col = None
        name = 'rangedata'
    else:
        index_col = 0
        name = 'envdata'
        
    df = pd.read_table(envfile, sep='\t', index_col=index_col)
    print(op.basename(envfile))
    display(df)
    
    envdfs[name] = df

tutorial_envfile_GFready_pooled.txt


,ISO,TSsd,PSsd,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,0.009381,-2.245441,1.003559,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
2,0.239071,-1.967663,0.518946,0.070353,-0.265649,0.799727,0.169512,-0.417965,0.315295
3,0.580610,-0.963343,-0.246231,0.264633,-0.248372,0.375248,-0.172690,-0.586408,-0.320803
4,0.639031,-0.164371,-0.399267,0.371616,0.167485,0.063118,-0.465978,-0.656053,-0.625240
5,-0.036985,-0.291426,-0.661614,0.244613,0.117437,-0.010939,-0.291486,-0.528748,-0.537376
...,...,...,...,...,...,...,...,...,...
96,0.113241,-0.239327,-0.054937,-0.288220,0.261489,-0.317123,-0.018548,0.228030,-0.045559
97,0.881061,0.598108,1.480884,-0.116570,0.323553,-0.231351,-0.269762,0.344912,0.059459
98,1.587251,1.669095,2.461321,0.093915,0.400397,-0.150975,-0.408936,0.588880,0.274592
99,1.381814,2.462649,2.880066,0.034177,0.399089,-0.440848,-0.464035,0.673533,0.360201


tutorial_rangefile_GFready_pooled.txt


,ISO,TSsd,PSsd,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM,lat,lon
0,0.009381,-2.245441,1.003559,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569,1.0,1.0
1,0.239071,-1.967663,0.518946,0.070353,-0.265649,0.799727,0.169512,-0.417965,0.315295,1.0,2.0
2,0.580610,-0.963343,-0.246231,0.264633,-0.248372,0.375248,-0.172690,-0.586408,-0.320803,1.0,3.0
3,0.639031,-0.164371,-0.399267,0.371616,0.167485,0.063118,-0.465978,-0.656053,-0.625240,1.0,4.0
4,-0.036985,-0.291426,-0.661614,0.244613,0.117437,-0.010939,-0.291486,-0.528748,-0.537376,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
95,0.113241,-0.239327,-0.054937,-0.288220,0.261489,-0.317123,-0.018548,0.228030,-0.045559,10.0,6.0
96,0.881061,0.598108,1.480884,-0.116570,0.323553,-0.231351,-0.269762,0.344912,0.059459,10.0,7.0
97,1.587251,1.669095,2.461321,0.093915,0.400397,-0.150975,-0.408936,0.588880,0.274592,10.0,8.0
98,1.381814,2.462649,2.880066,0.034177,0.399089,-0.440848,-0.464035,0.673533,0.360201,10.0,9.0


In [7]:
# compare to original that was used in /home/b.lind/offsets/tutorial_shuffle_run_20220919
og_envdata = pd.read_table(
    '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_files/tutorial_envfile_GFready_pooled.txt',
    index_col=0
)

og_envdata

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
2,0.070353,-0.265649,0.799727,0.169512,-0.417965,0.315295
3,0.264633,-0.248372,0.375248,-0.172690,-0.586408,-0.320803
4,0.371616,0.167485,0.063118,-0.465978,-0.656053,-0.625240
5,0.244613,0.117437,-0.010939,-0.291486,-0.528748,-0.537376
...,...,...,...,...,...,...
96,-0.288220,0.261489,-0.317123,-0.018548,0.228030,-0.045559
97,-0.116570,0.323553,-0.231351,-0.269762,0.344912,0.059459
98,0.093915,0.400397,-0.150975,-0.408936,0.588880,0.274592
99,0.034177,0.399089,-0.440848,-0.464035,0.673533,0.360201


In [8]:
# assert that the values for the adaptive envs are the same as above
for env in og_envdata.columns:
    assert all(og_envdata[env] == envdfs['envdata'][env])
    
    copy = envdfs['rangedata'][env].copy()
    copy.index = og_envdata.index
    assert all(og_envdata[env] == copy)

In [9]:
og_envdata.shape

(100, 6)

In [10]:
envdfs['rangedata'].shape

(100, 11)

In [11]:
# symlink the shuffled+nuis envfiles
for src in envfiles:
    basename = op.basename(src)
    
    dst = f'{gf_training_dir}/{basename}'
    
    if not op.exists(dst):
        os.symlink(src, dst)
        
    print(src)
    print('\t', dst)

/home/b.lind/offsets/tutorial_shuffle_nuisance_run_20220919/gradient_forests/training/training_files/tutorial_envfile_GFready_pooled.txt
	 /home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_files/tutorial_envfile_GFready_pooled.txt
/home/b.lind/offsets/tutorial_shuffle_nuisance_run_20220919/gradient_forests/training/training_files/tutorial_rangefile_GFready_pooled.txt
	 /home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_files/tutorial_rangefile_GFready_pooled.txt


# Get SNP data

symlink snp files from normal evaluation of multivariate sims

In [12]:
# get unshuffled allele freqs for the multivariate simss
og_snpdir = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_files'

snpfiles = fs(og_snpdir, pattern='-gt-')

[op.basename(f) for f in snpfiles]

['tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_adaptive.txt',
 'tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all.txt',
 'tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_neutral.txt']

In [13]:
# create symlinks
for src in snpfiles:
    basename = op.basename(src)
    
    dst = f'{gf_training_dir}/{basename}'
    
    if not op.exists(dst):
        os.symlink(src, dst)

# create shfiles


In [14]:
src_shdir = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_shfiles'
src_shfiles = fs(src_shdir, endswith='.sh')

src_shfiles

['/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_shfiles/tutorial_GF_training_pooled_adaptive.sh',
 '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_shfiles/tutorial_GF_training_pooled_all.sh',
 '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_shfiles/tutorial_GF_training_pooled_neutral.sh',
 '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_shfiles/tutorial_gf_training_watcher.sh']

In [15]:
nuis_dir

'/home/b.lind/offsets/run_20220919_nuisance/tutorial'

In [16]:
new_shfiles = []
for sh in src_shfiles:
    text = read(sh, lines=False)
    
    job = op.basename(sh).split(".")[0]
    new_job = job.replace('_GF_', '_nuis_GF_')
    
    text = text.replace('/home/b.lind/offsets/run_20220919_tutorial', nuis_dir
                       ).replace(job, new_job)
    
    new_sh = op.join(gf_shdir, f'{new_job}.sh')
    with open(new_sh, 'w') as o:
        o.write(text)

    new_shfiles.append(new_sh)
    
print(text)

#!/bin/bash
#SBATCH --job-name=tutorial_gf_training_watcher
#SBATCH --time=0:00:01
#SBATCH --ntasks=1
#SBATCH --mem=25
#SBATCH --output=tutorial_gf_training_watcher_%j.out
#SBATCH --dependency=afterok:34315837,34315838,34315839
#SBATCH --mail-user=b.lind@northeastern.edu
#SBATCH --mail-type=FAIL
#SBATCH --mail-type=END








In [17]:
# make the outdir
makedir('/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_outfiles')

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_outfiles'

In [18]:
new_shfiles

['/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_shfiles/tutorial_nuis_GF_training_pooled_adaptive.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_shfiles/tutorial_nuis_GF_training_pooled_all.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_shfiles/tutorial_nuis_GF_training_pooled_neutral.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_shfiles/tutorial_gf_training_watcher.sh']

In [19]:
# sbatch the shfiles
pids = sbatch(new_shfiles[:-1])

sbatching: 100%|███████████████| 3/3 [00:00<00:00,  6.68it/s]


In [20]:
new_shfiles[-1]

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_shfiles/tutorial_gf_training_watcher.sh'

In [21]:
# update the dependency pids in the watcher file
watcher_sh = new_shfiles[-1]

text = read(watcher_sh, lines=True)
assert 'dependency' in text[6]
text[6] = '#SBATCH --dependency=afterok:%s' % ','.join(pids)

with open(watcher_sh, 'w') as o:
    o.write('\n'.join(text))

In [22]:
sbatch(watcher_sh)

sbatching: 100%|███████████████| 1/1 [00:00<00:00,  7.22it/s]


['36539022']

In [23]:
formatclock(dt.now() - t0)

'0-00:00:01'

# check on progress

In [1]:
from pythonimports import *

import MVP_summary_functions as mvp

t0 = dt.now()  # notebook timer

nuis_dir = makedir('/home/b.lind/offsets/run_20220919_nuisance/tutorial')

gf_training_dir = makedir(f'{nuis_dir}/gradient_forests/training/training_files')
gf_shdir = makedir(gf_training_dir.replace('_files', '_shfiles'))
gf_outdir = makedir(gf_shdir.replace('_shfiles', '_outfiles'))

mvp.latest_commit()
session_info.show()

#########################################################
Today:	June 02, 2023 - 10:45:41
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit c721bd0d89c1e150e17a4e0739c26e4296405233  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Wed May 10 15:13:03 2023 -0400

Current commit of MVP_offsets:
commit 060db40dff8355d2087deab28bd695fcccee8f49  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu May 18 12:50:56 2023 -0400
#########################################################



In [2]:
shfiles = fs(gf_shdir, endswith='.sh')
outs = fs(gf_shdir, endswith='.out')

len(shfiles), len(outs)

(4, 4)

In [3]:
seffs = Seffs(outs=outs)

seffs.states.counter()

executing seff commands: 100%|███████████████| 4/4 [00:00<00:00,  4.62it/s]


Counter({'State: COMPLETED (exit code 0)': 4})

In [5]:
[op.basename(sh) for sh in seffs.shfiles]

['tutorial_gf_training_watcher.sh',
 'tutorial_nuis_GF_training_pooled_adaptive.sh',
 'tutorial_nuis_GF_training_pooled_all.sh',
 'tutorial_nuis_GF_training_pooled_neutral.sh']